# Install the required libraries

In [ ]:
!pip install -q hvplot

In [ ]:
pip install typing-extensions --upgrade

In [ ]:
!pip install hvplot

In [ ]:
!pip install GitPython==3.0.7

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
%matplotlib inline
sns.set_style("darkgrid")

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more = pd.read_csv("ResTracker Export after preprocessing_V1.csv")
del DHP_rename_Remove_DHP_Case_Drop_more[DHP_rename_Remove_DHP_Case_Drop_more.columns[0]]

In [ ]:
X = DHP_rename_Remove_DHP_Case_Drop_more[['Gas Date', 'Gas Rate(TJ/d)', 'Water Rate(m3/d)', 'Lat','Long','Cum Gas(bcf)', 'Cum Water(10e3 m3)', 'Cum Online Days',
                 'CP(kPag)','Depth(m)']]
y = DHP_rename_Remove_DHP_Case_Drop_more['DHP(psi)']

# Visualize the variable histogram distribution

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist("Water Rate(m3/d)")

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist("Gas Rate(TJ/d)")

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist("Lat")

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist("Long")

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist('Cum Gas(bcf)')

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist('Cum Water(10e3 m3)')

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist('Cum Online Days')

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.hist('CP(kPag)')

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.scatter(x='Gas Rate(TJ/d)', y='DHP(psi)')

In [ ]:
DHP_rename_Remove_DHP_Case_Drop_more.hvplot.scatter(x='Depth(m)', y='DHP(psi)')

# Split the test and training dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

# Standardize the dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# Build the ANN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential()

model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer=Adam(0.01), loss='mse')

r = model.fit(X_train, y_train,
              validation_data=(X_test,y_test),
              batch_size=256,
              epochs=100)

In [ ]:
y_test
test_pred
test_pred_array = test_pred.ravel()
test_pred_array

In [ ]:
pd.DataFrame({'True Values': y_test, 'Predicted Values': test_pred_array }).hvplot.scatter(x='True Values', y='Predicted Values')

# Loss evaluation

In [ ]:
pd.DataFrame(r.history)

In [ ]:
pd.DataFrame(r.history).hvplot.line(y=['loss', 'val_loss'])

In [ ]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

# Model accuracy evalution

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Artficial Neural Network", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)

In [ ]:
model.summary()

# Tensorflow modle save

In [ ]:
tensorflow.saved_model.save(model,"C:/Users/fren/Desktop/ANN Models")

In [ ]:
tensorflow.saved_model.load("C:/Users/fren/Desktop/ANN Models")

# Import and standard scaler the prediction dataset

In [ ]:
DHP_Pred_Dataset = pd.read_csv('ResTracker Export_Filtered_ANN_DHP_prediction_V1.csv')

In [ ]:
DHP_Pred_Dataset_rename = DHP_Pred_Dataset.rename(columns={'Latitude':'Lat','Longitude':'Long','Gas Rate (TJ/d)':'Gas Rate(TJ/d)', 'Water Rate (m3/d)':'Water Rate(m3/d)','Cumulative Gas(bcf)':'Cum Gas(bcf)','Cumulative Water(10e3 m3)':'Cum Water(10e3 m3)','Cumulative Online Days':'Cum Online Days','Casing Pressure (kPag)':'CP(kPag)','Downhole Pressure (psi)':'DHP(psi)','Depth_Argle_m':'Depth(m)'})

In [ ]:
DHP_Pred_Dataset_Pre = DHP_Pred_Dataset_rename [['Gas Date', 'Gas Rate(TJ/d)', 'Water Rate(m3/d)','Lat','Long', 'Cum Gas(bcf)', 'Cum Water(10e3 m3)', 'Cum Online Days',
                 'CP(kPag)','Depth(m)']]

In [ ]:
import numpy as np
scaler = StandardScaler()
DHP_Prediction = scaler.fit_transform (DHP_Pred_Dataset_Pre)

In [ ]:
DHP_Pred = model.predict(DHP_Prediction).ravel()

In [ ]:
DHP_Pred

# Obtain the predicted DHP values

In [ ]:
DHP_Pred = pd.DataFrame(DHP_Pred)
DHP_Pred.to_csv("output_for_DHP_V1.CSV")